In [10]:
import boto3

import rasterio as rio
import numpy as np

import os
import sys
import threading

In [ ]:
# Aquastat foo.028

In [12]:
# local storage

merge_file_fao = "/Users/nathansuberi/Desktop/RW_Data/merging_aquastat.tif"

# S3 storage
s3_bucket_remote_read = "s3://wri-public-data/"
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/foo_028_aquastat_irrigated_areas/"

s3_file1 = "foo_028_aquastat_irrigated_areas_area_equipped_irrigation_edit.tif"
s3_file2 = "foo_028_aquastat_irrigated_areas_aai_pct_aei_edit.tif"
s3_file3 = "foo_028_aquastat_irrigated_areas_aeigw_pct_aei_edit.tif"
s3_file4 = "foo_028_aquastat_irrigated_areas_aeisw_pct_aei_edit.tif"
s3_file5 = "foo_028_aquastat_irrigated_areas_aeinc_pct_aei_edit.tif"

merge_files = [s3_bucket_remote_read + s3_folder + s3_file1, 
               s3_bucket_remote_read + s3_folder + s3_file2, 
               s3_bucket_remote_read + s3_folder + s3_file3, 
               s3_bucket_remote_read + s3_folder + s3_file4, 
               s3_bucket_remote_read + s3_folder + s3_file5]
band_names = ["area equipped with irrigation", 
              "area actually irrigated",
              "percent of irrigation equipped area using groundwater",
              "percent of irrigation equipped area using surface water",
              "percent of irrigation equipped area using nonconventional water sources",
             ]

s3_key_merge = s3_folder + "foo_028_aquastat_irrigated_areas_merged.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [8]:
# for file in merge_files: 
#     with rio.open(file, 'r') as src:
#         print(src.profile)
        
with rio.open(merge_files[0], 'r') as src:
    profile = src.profile
    
profile.update(count=len(merge_files), band_names=band_names)

In [9]:
with rio.open(merge_file_fao, "w", **profile) as dst:
    for ix, file in enumerate(merge_files):
        with rio.open(file,'r') as src:
            src_data = src.read(indexes=1)
            dst_band = ix+1
            dst.write(src_data, indexes=dst_band)`

In [13]:
s3_upload.upload_file(merge_file_fao, s3_bucket, s3_key_merge,
           Callback=ProgressPercentage(merge_file_fao))

/Users/nathansuberi/Desktop/RW_Data/merging_aquastat.tif  11591196 / 11591196.0  (100.00%)

In [14]:
os.environ["s3_key"] = "s3://wri-public-data/" + s3_key_merge
os.environ["gs_key"] = "gs://resource-watch-public/" + s3_key_merge

!gsutil cp $s3_key $gs_key

Non-MD5 etag ("7d605e38dd21b804e599fb863c662260-2") present for key <Key: wri-public-data,resourcewatch/raster/foo_028_aquastat_irrigated_areas/foo_028_aquastat_irrigated_areas_merged.tif>, data integrity checks are not possible.
Copying s3://wri-public-data/resourcewatch/raster/foo_028_aquastat_irrigated_areas/foo_028_aquastat_irrigated_areas_merged.tif [Content-Type=binary/octet-stream]...
- [1 files][ 11.1 MiB/ 11.1 MiB]                                                
Operation completed over 1 objects/11.1 MiB.                                     


In [23]:
os.environ["asset_id"] = "users/resourcewatch/foo_028_aquastat_irrigated_areas"

!earthengine upload image --asset_id=$asset_id $gs_key

Started upload task with ID: 5K3A2FL7YLPLWRLZWNDO5OEB


In [16]:
!earthengine acl set public $asset_id
os.environ["band_names"] = str(band_names)
!earthengine asset set -p '(string)band_names='"$band_names" $asset_id

In [ ]:
# Soc.030

In [18]:
# local storage

merge_file_fao = "/Users/nathansuberi/Desktop/RW_Data/merging_aquastat.tif"

# S3 storage
s3_bucket_remote_read = "s3://wri-public-data/"
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/soc_030_gross_domestic_product/"

band_names = ['mer_1990', 'mer_1995', 'mer_2000', 'mer_2005', 'ppp_1990', 'ppp_1995', 'ppp_2000', 'ppp_2005']
s3_key_merge = s3_folder + "soc_030_gross_domestic_product.tif"

In [19]:
with rio.open(s3_bucket_remote_read + s3_key_merge, 'r') as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': -9999.0, 'width': 360, 'height': 140, 'count': 8, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(1.0, 0.0, -180.0,
       0.0, -1.2857142857142858, 90.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}


In [20]:
os.environ["s3_key"] = "s3://wri-public-data/" + s3_key_merge
os.environ["gs_key"] = "gs://resource-watch-public/" + s3_key_merge

!gsutil cp $s3_key $gs_key

Copying s3://wri-public-data/resourcewatch/raster/soc_030_gross_domestic_product/soc_030_gross_domestic_product.tif [Content-Type=binary/octet-stream]...
- [1 files][882.4 KiB/882.4 KiB]                                                
Operation completed over 1 objects/882.4 KiB.                                    


In [21]:
os.environ["asset_id"] = "users/resourcewatch/soc_030_gross_domestic_product"

!earthengine upload image --asset_id=$asset_id $gs_key

Started upload task with ID: UAP6UN3A4CSWMTSKOM3XQNZZ


In [22]:
!earthengine acl set public $asset_id
os.environ["band_names"] = str(band_names)
!earthengine asset set -p '(string)band_names='"$band_names" $asset_id